In [1]:
import pandas as pd
import numpy as np
import pathlib
import plotly
import geopandas
from pathlib import Path
import re
import os

import plotly.express as px
import plotly.graph_objects as go

## Group snowgun production
- Saetde Data
- Classified by snowgun production

In [2]:
path_to_folders = r'C:\Users\montse\Desktop\ARI-Git\jupyterlab-data-science\src\data\canons_saetde'

def load_csvs_from_folder(path_to_folders, file_name='Consommations eau par abri.csv', name='arqueta', sort=True, join='outer'):
    """Carregar els csv amb el nom file_name de tots els subfolders de la path_to_folders
    Return a pd.Dataframe with all files as a column
    """
    
    df_ls = []
    for path in Path(path_to_folders).iterdir():
        if path.is_dir():
            p = path / file_name
            df = pd.read_csv(p, skiprows=4, delimiter=';', index_col=0, names=[name, Path(p).parts[-2]], usecols=[0, 1])
            df_ls.append(df)
        
    return pd.concat(df_ls, axis=1, sort=sort, join=join).reset_index().rename(columns={'index': name})

In [3]:
consum_aigua = load_csvs_from_folder(path_to_folders)

#Drop unwanted cumulative rows
consum_aigua.drop(consum_aigua[( (consum_aigua.arqueta.str.contains('Cumul')) | (consum_aigua.arqueta.str.contains('XM')) ) ].index, inplace=True)

#Define sg_code for each snowgun
def pick_numeric_values(string):
    return re.sub("[^0-9]", "", string)
consum_aigua['sg_code'] = consum_aigua.apply(lambda row: pick_numeric_values(row['arqueta']), axis = 1).astype('str')

#Define the category type for each snowgun
consum_aigua['category'] = consum_aigua['arqueta'].apply(lambda x: x[:2])
consum_aigua = consum_aigua[consum_aigua.category.isin(['YA', 'YT'])]
consum_aigua = consum_aigua.fillna(value=0)
#print(consum_aigua.to_string())

In [4]:
path_geo_data_saetde =  r'C:\Users\montse\Desktop\ARI-Git\jupyterlab-data-science\src\data\geo_data_saetde\vectorial_data_saetde.gpkg'
saetde_geo = geopandas.read_file(path_geo_data_saetde)
#print(saetde_geo.to_string())
#print(saetde_geo.dtypes)

saetde_geo['sg_code'] = saetde_geo['sg_code'].astype('str') #Convert sg_code column into the right format
saetde_snow_production = pd.merge(consum_aigua, saetde_geo.drop('category', axis=1), how='inner', on='sg_code')
saetde_snow_production = geopandas.GeoDataFrame(saetde_snow_production)
saetde_snow_production = saetde_snow_production.to_crs("EPSG:4326")

#As the only columns that are ints or floats are the ones containing the snow production, we can sum up those columns
  #print(saetde_snow_production.dtypes.to_string())
saetde_snow_production['year_tot_production'] = saetde_snow_production[list(saetde_snow_production.columns)].sum(axis=1)
#print(saetde_snow_production.head(1))

C:\Users\montse\AppData\Local\Temp\ipykernel_15720\2972473442.py:13: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  saetde_snow_production['year_tot_production'] = saetde_snow_production[list(saetde_snow_production.columns)].sum(axis=1)


In [19]:
saetde_snow_reduced = saetde_snow_production[['trail_code', 'customer', 'ski_resort', 'sector', 'geometry', 'year_tot_production']].copy()
saetde_snow_reduced.to_file('./outputs/shapefile_geojson/snow_production/snow_production.shp')

C:\Users\montse\AppData\Local\Temp\ipykernel_15720\3111763937.py:2: UserWarning:

Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.



In [6]:
mapbox_token = os.getenv('mapbox_token')

px.set_mapbox_access_token(mapbox_token)
map_snowprod = px.scatter_mapbox(
  saetde_snow_production, 
  lat=saetde_snow_production.geometry.y,
  lon=saetde_snow_production.geometry.x,
  hover_name= 'trail_code',
  zoom= 12.7,
  color=saetde_snow_production['year_tot_production'], 
  color_discrete_sequence=px.colors.qualitative.G10,
  size=1 + 80*saetde_snow_production.year_tot_production,
  opacity=0.6,
  size_max=12,
  width=1100,
  height=810,
  center={'lat': 42.538, 'lon': 1.70}, 
  title='Snow production for each snowgun, year 2021',

  mapbox_style= 'carto-darkmatter'
  #'satellite'
)
map_snowprod.show()

In [7]:
snow_saetde = saetde_snow_production.groupby(by= saetde_snow_production.trail_code).agg('sum')
snow_saetde = pd.DataFrame({'tot_production' : snow_saetde['year_tot_production']})

snow_prod_chart = px.bar(
  x=snow_saetde.index,
  y=snow_saetde['tot_production'],
  color=snow_saetde['tot_production'],
  color_discrete_sequence= px.colors.sequential.Plasma,
  title= 'Snow Production in the different Saetde slopes'
)
snow_prod_chart.show()